Bước A — Chuẩn bị DB (nhanh nhất với SQLite)

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# 1) Nạp CSV
df = pd.read_csv("retail_store_inventory.csv")

# 2) Tạo SQLite & ghi bảng
engine = create_engine("sqlite:///inventory.db")
df.to_sql("inventory", engine, if_exists="replace", index=False)


73100

Bước B — Kết nối SQLDatabase (LangChain)

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///inventory.db")
print(db.get_usable_table_names())   # -> {'inventory'}
print(db.get_table_info(["inventory"])) # xem schema để LLM dùng


['inventory']

CREATE TABLE inventory (
	"Date" TEXT, 
	"Store ID" TEXT, 
	"Product ID" TEXT, 
	"Category" TEXT, 
	"Region" TEXT, 
	"Inventory Level" BIGINT, 
	"Units Sold" BIGINT, 
	"Units Ordered" BIGINT, 
	"Demand Forecast" FLOAT, 
	"Price" FLOAT, 
	"Discount" BIGINT, 
	"Weather Condition" TEXT, 
	"Holiday/Promotion" BIGINT, 
	"Competitor Pricing" FLOAT, 
	"Seasonality" TEXT
)

/*
3 rows from inventory table:
Date	Store ID	Product ID	Category	Region	Inventory Level	Units Sold	Units Ordered	Demand Forecast	Price	Discount	Weather Condition	Holiday/Promotion	Competitor Pricing	Seasonality
2022-01-01	S001	P0001	Groceries	North	231	127	55	135.47	33.5	20	Rainy	0	29.69	Autumn
2022-01-01	S001	P0002	Toys	South	204	150	66	144.04	63.01	20	Sunny	0	66.16	Autumn
2022-01-01	S001	P0003	Toys	West	102	65	51	74.02	27.99	10	Sunny	1	31.32	Summer
*/


In [3]:
# thêm OPENAI_API_KEY vào biến môi trường trước khi chạy
OPENROUTER_API_KEY="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"  # dán API key OpenRouter thật vào đây
import os
os.environ["OPENAI_API_KEY"] = OPENROUTER_API_KEY

In [23]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import (
    QuerySQLDataBaseTool, InfoSQLDatabaseTool, ListSQLDatabaseTool
)
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

db = SQLDatabase.from_uri("sqlite:///inventory.db")

tools = [
    QuerySQLDataBaseTool(db=db),
    InfoSQLDatabaseTool(db=db),
    ListSQLDatabaseTool(db=db),
]

llm = ChatOpenAI(
    # model="openai/gpt-oss-20b:free",  # hoặc openai/gpt-oss-20b:pro
    model="google/gemma-3-12b-it:free",                 # hoặc deepseek/deepseek-chat:free
    openai_api_base="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    temperature=0,
)

prompt = hub.pull("hwchase17/react")               # prompt ReAct chuẩn

react_agent = create_react_agent(llm, tools, prompt)
executor = AgentExecutor(agent=react_agent, tools=tools, verbose=True)

try:
    print(executor.invoke({"input": "Top 5 Category có Units Sold cao nhất toàn hệ thống?"}))
except Exception as e:
    print(e)


c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

I need to find the top 5 categories with the highest total units sold across the entire system. This requires querying the database to sum the 'Units Sold' for each category and then ordering the results in descending order, finally limiting the output to the top 5.

Action: sql_db_query
Action Input: "SELECT Category, SUM(Units_Sold) AS TotalUnitsSold FROM Sales GROUP BY Category ORDER BY TotalUnitsSold DESC LIMIT 5;"Error: (sqlite3.OperationalError) no such table: Sales
[SQL: SELECT Category, SUM(Units_Sold) AS TotalUnitsSold FROM Sales GROUP BY Category ORDER BY TotalUnitsSold DESC LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)The error indicates that the table 'Sales' does not exist. I need to list the tables in the database to identify the correct table name.

Action: sql_db_list_tables
Action Input: ""inventory
The database contains only one table named 'inventory'. I need to examine the schema of the 'inventory' ta